In [9]:
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D

In [11]:
# Constants
DATA_DIR = 'data/'
CSV_FILE = 'driving_log.csv'

# Parameters
steering_correction = 0.25

In [12]:
# Load csv log file
lines = []
with open(DATA_DIR+CSV_FILE) as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # skip the headers
    for line in reader:
        lines.append(line)

In [13]:
# Load images and steering data

#Cameras parameters
cameras = ['center', 'left', 'right']
cameras_index = {'center':0, 'left':1, 'right':2} # 0:center, 1:left, 2:right
cameras_steering_corretion = {'center':0, 'left':steering_correction, 'right':-steering_correction}

# Data containers
images = []
measurements = []

# Uploading images
for line in lines:
    for camera in cameras:
        source_path = DATA_DIR + line[cameras_index[camera]].strip()
        image = cv2.imread(source_path)
        images.append(image)
        measurement = float(line[3])+cameras_steering_corretion[camera]
        measurements.append(measurement)
X_data = np.array(images)
y_data = np.array(measurements)

In [14]:
# Define simple model
model = Sequential()
model.add(Cropping2D(cropping=((30,25),(0,0)), input_shape = (160, 320, 3)))
model.add(Lambda(lambda x: x / 255.0 - 0.5))
model.add(Flatten())
model.add(Dense(1))

# Train simple model
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_data, y_data, validation_split = 0.2, shuffle = True, epochs = 3)

# Save model
model.save('model.h5')

Train on 19286 samples, validate on 4822 samples
Epoch 1/3
19286/19286 [==============================] - 65s - loss: 1.3113 - val_loss: 3.6547
Epoch 2/3
19286/19286 [==============================] - 50s - loss: 1.7745 - val_loss: 1.0969
Epoch 3/3
19286/19286 [==============================] - 50s - loss: 1.8397 - val_loss: 1.7616
